Read data file.

In [ ]:
import pandas as pd

In [ ]:
data = pd.read_csv("./data.csv", header=None)

In [ ]:
data

Make empty columns.

In [ ]:
data[1] = ""
data[2] = ""
data[3] = ""
data[4] = ""
data[5] = ""

Parsing data. Dirty trick, but it's working.

In [ ]:
import re

In [ ]:
for i in range(0, len(data)):
    row = re.split(r';', data[0][i])
    rowLen = len(row)
    if rowLen >= 1:
        dateTime = re.findall(r"^(\d{4}\-\d{2}\-\d{2}\s\d{2}:\d{2}:\d{2})", row[0])
        if len(dateTime) >=1:
            data[1][i] = dateTime[0]
    if rowLen >= 2:
        data[2][i] = row[1]
    if rowLen >= 3:
        data[3][i] = row[2]
    if rowLen >= 4:
        data[4][i] = row[3]
    if rowLen >= 5:
        data[5][i] = row[4]

Get separate values in separate data frames.

In [ ]:
v = []
for i in range(0, len(data)):
    if data[2][i] == '"voltage"':
        val = re.findall(r"(\d{0,}\.\d{0,})", data[3][i])
        v.append({"date": data[1][i], "value": val[0]})
voltage = pd.DataFrame(v)

In [ ]:
voltage

In [ ]:
v = []
for i in range(0, len(data)):
    if data[2][i] == '"moisture"':
        val = re.findall(r"(\d{0,}\.\d{0,})", data[3][i])
        v.append({"date": data[1][i], "value": val[0]})
moisture = pd.DataFrame(v)

In [ ]:
moisture

In [ ]:
v = []
for i in range(0, len(data)):
    if data[2][i] == '"water"':
        val = re.findall(r"(\d{0,}\.\d{0,})", data[3][i])
        v.append({"date": data[1][i], "value": val[0]})
water = pd.DataFrame(v)

In [ ]:
water

Combining data frames into one data frame.

In [ ]:
import datetime as dt

In [ ]:
def getDate(value):
    parse =  re.findall(r"\d{4}|\d{2}", value)
    return dt.datetime(int(parse[0]), int(parse[1]), int(parse[2]), int(parse[3]), int(parse[4]), int(parse[5]))
def compareDates(date1, date2):
    return int((getDate(date1) - getDate(date2)).total_seconds())

In [ ]:
maxlen = max(len(voltage), len(moisture), len(water))
i = 0
j = 0
k = 0
result = []    
while(True):
    dif_1 = compareDates(voltage['date'][i], moisture['date'][j])
    dif_2 = compareDates(moisture['date'][j], water['date'][k])
    if(dif_1 <= 1 and dif_1 >= -1):
        if(dif_2 <= 1 and dif_2 >= -1):
            result.append({"date": getDate(voltage['date'][i]), "voltage": voltage['value'][i], "moisture": moisture['value'][j], "water": water['value'][k]})
            k = k + 1
        else:
            result.append({"date": getDate(voltage['date'][i]), "voltage": voltage['value'][i], "moisture": moisture['value'][j]})
        i = i + 1
        j = j + 1
    else:
        if(dif_1 > 0):
            j = j + 1
        else:
            i = i + 1
    if(i >= maxlen or j >= maxlen or k >= maxlen):
        break
output = pd.DataFrame(result)

In [ ]:
output

Delete rows where time distance between measurements less than 55 minutes

In [ ]:
def compareDatesA(date1, date2):
    return int((date1 - date2).total_seconds())

In [ ]:
toDrop = []
for i in range(0, len(output) - 1):
    if compareDatesA(output["date"][i + 1], output["date"][i]) < 3300:
        toDrop.append(i)
output.drop(toDrop, inplace = True)

In [ ]:
output

Save final data to excel file

In [ ]:
output.to_excel("./output.xlsx")